In [0]:
driver="com.microsoft.sqlserver.jdbc.SQLServerDriver"
database_host="sqldb-prod-east.database.windows.net"
database_port="1433"
database_name="db_Mars"

connection_properties = {
    "user": "marssqladmin",
    "password": "2FQdcntxIIx$RoPQz21y",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
url = f"jdbc:sqlserver://{database_host}:{database_port};database={database_name}"

Bring over updated sql staging_cdc.stg_... tables from db_mars

In [0]:
def load_staging(cfg):
    """
    Reads the SQL CDC staging table and writes it out to the databricks staging table of that capture instance
    """
    df = (
      spark.read
           .jdbc(url=url,
                 table=cfg.Sql_SourceStagingTable,
                 properties=connection_properties)
    )
    (df.write
       .format("delta")
       .mode("overwrite")
       .option("overwriteSchema", "true")
       .saveAsTable(cfg.StagingTable)
    )
    print(f" Loaded staging table {cfg.StagingTable}")


Apply Updates to bronze tables

In [0]:
from pyspark.sql import Row

def apply_cdc(cfg: Row):
    """
    Given one row of Test_CDC_Staging_Config, run MERGE+DELETE to keep bronze Destination in sync with azure sql source tables.
    """
    sql_staging = cfg.Sql_SourceStagingTable
    staging = cfg.StagingTable
    target  = cfg.DestinationTable
    cols    = [c.strip() for c in cfg.ColumnList.split(",")]
    pk      = cols[0]

    update_set = ",\n        ".join(f"{c} = src.{c}" for c in cols)
    insert_cols = ", ".join(cols)
    insert_vals = ", ".join(f"src.{c}" for c in cols)

    merge_sql = f"""
    MERGE INTO {target} AS tgt
    USING (
      SELECT * FROM {staging} WHERE OperationCode IN (2,4)
    ) AS src
      ON tgt.{pk} = src.{pk}

    WHEN MATCHED THEN
      UPDATE SET
        {update_set}

    WHEN NOT MATCHED THEN
      INSERT ({insert_cols})
      VALUES ({insert_vals})
    """

    delete_sql = f"""
    DELETE FROM {target}
    WHERE {pk} IN (
      SELECT {pk} FROM {staging} WHERE OperationCode = 1
    )
    """

    spark.sql(merge_sql)
    spark.sql(delete_sql)
    print(f"synced {target}")


Run functions to bring over staging data and merge

In [0]:
#cfg_df = spark.table("staging_cdc.pipeline_config").filter("IsActive")
#for cfg in cfg_df.collect():
    #load_staging(cfg)
    #apply_cdc(cfg)